In [2]:
import pandas as pd
import igraph as ig
from ipysigma import Sigma
import glob
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

# Comparison on three iteration

In [3]:
edgelist_incel = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./incel*/*_edgelist/*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_incel = pd.concat([edgelist_incel,df])

In [4]:
edgelist_incel['link'] = ['incel']*len(edgelist_incel)

In [5]:
edgelist_right = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./*right*/*_edgelist/*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_right = pd.concat([edgelist_right,df])

In [6]:
edgelist_right['link'] = ['right']*len(edgelist_right)

In [7]:
edgelist_left = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./*left*/*_edgelist/*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_left = pd.concat([edgelist_left,df])

In [8]:
edgelist_left['link'] = ['left']*len(edgelist_left)

In [9]:
edgelist = pd.concat([edgelist_incel, edgelist_left, edgelist_right])

In [10]:
with open('./incel_network/dictionaires/id_to_desc.json') as file_desc,\
    open('./incel_network/dictionaires/id_to_names.json') as file_names:
    id_to_desc_incel = json.load(file_desc)
    id_to_names_incel = json.load(file_names)

In [11]:
with open('./extreme_right_network/dictionaires/id_to_desc.json') as file_desc,\
    open('./extreme_right_network/dictionaires/id_to_names.json') as file_names:
    id_to_desc_right = json.load(file_desc)
    id_to_names_right = json.load(file_names)

In [12]:
with open('./extreme_left_network/dictionaires/id_to_desc.json') as file_desc,\
    open('./extreme_left_network/dictionaires/id_to_names.json') as file_names:
    id_to_desc_left = json.load(file_desc)
    id_to_names_left = json.load(file_names)

In [13]:
id_to_desc_incel = dict(map(lambda x: (int(x[0]), x[1]), id_to_desc_incel.items()))
id_to_desc_right = dict(map(lambda x: (int(x[0]), x[1]), id_to_desc_right.items()))
id_to_desc_left = dict(map(lambda x: (int(x[0]), x[1]), id_to_desc_left.items()))

id_to_names_incel = dict(map(lambda x: (int(x[0]), x[1]), id_to_names_incel.items()))
id_to_names_right = dict(map(lambda x: (int(x[0]), x[1]), id_to_names_right.items()))
id_to_names_left = dict(map(lambda x: (int(x[0]), x[1]), id_to_names_left.items()))

In [14]:
id_to_names = {**id_to_names_incel, **id_to_names_right, ** id_to_names_left}
id_to_desc = {**id_to_desc_incel, **id_to_desc_right, **id_to_desc_left}

## Visualization

### Incel and extreme right

In [14]:
edgelist_ir = edgelist[(edgelist['link']=='incel') | (edgelist['link']=='right')]

In [15]:
G = ig.Graph.TupleList(edgelist_ir.iloc[:,:3].itertuples(index=False), directed=True, weights=True)

In [16]:
## community detection
#communities = G.as_undirected().community_leiden(objective_function="modularity")
communities = G.community_walktrap(weights= G.es['weight'], steps=4)
# add community as node label
G.vs['community'] = communities.as_clustering().membership

In [17]:
G.vs['desc'] = list(map(lambda x: id_to_desc.get(int(x),x), G.vs['name']))
G.vs['label'] = list(map(lambda x: id_to_names.get(int(x),x), G.vs['name']))

In [18]:
Sigma(G, node_size=G.degree(mode='out'),
      node_color=G.vs['community'], 
      hide_edges_on_move=True,
      edge_color=edgelist_ir['link'].tolist(),
      node_color_gradient='Viridis', node_label=G.vs['label'],
     layout_settings = {"barnesHutOptimize":False,
                       'gravity': 0.1})

Sigma(ig.Graph with 6,206 nodes and 16,948 edges)

### incel and extreme left

In [21]:
edgelist_il = edgelist[(edgelist['link']=='incel') | (edgelist['link']=='left')]

In [22]:
G = ig.Graph.TupleList(edgelist_il.iloc[:,:3].itertuples(index=False), directed=True, weights=True)

In [23]:
## community detection
#communities = G.as_undirected().community_leiden(objective_function="modularity")
communities = G.community_walktrap(weights= G.es['weight'], steps=4)
# add community as node label
G.vs['community'] = communities.as_clustering().membership

In [24]:
G.vs['desc'] = list(map(lambda x: id_to_desc.get(int(x),x), G.vs['name']))
G.vs['label'] = list(map(lambda x: id_to_names.get(int(x),x), G.vs['name']))

In [25]:
Sigma(G, node_size=G.degree(mode='out'),
      node_color=G.vs['community'], 
      hide_edges_on_move=True,
      edge_color=edgelist['link'].tolist(),
      node_color_gradient='Viridis', node_label=G.vs['label'],
     layout_settings = {"barnesHutOptimize":False,
                       'gravity': 0.1})

Sigma(ig.Graph with 6,270 nodes and 17,424 edges)

## Comparison elaboration

### comparison on total degree

In [15]:
nodes_incel = edgelist[edgelist['link']=='incel'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_incel = nodes_incel.values.reshape((-1)) #flatten array

nodes_incel = np.unique(nodes_incel) #get unique values

In [16]:
nodes_right = edgelist[edgelist['link']=='right'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_right = nodes_right.values.reshape((-1)) #flatten array

nodes_right = np.unique(nodes_right) #get unique values

In [17]:
nodes_left = edgelist[edgelist['link']=='left'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_left = nodes_left.values.reshape((-1)) #flatten array

nodes_left = np.unique(nodes_left) #get unique values

filter nodes common to all three

In [18]:
nodes_step_1 = nodes_incel[np.isin(nodes_incel, nodes_right)] #get nodes of incel network that are ALSO in right network
common_nodes = nodes_step_1[np.isin(nodes_step_1, nodes_left)] #get nodes of intermediate step (present in both right and incel network) that are ALSO in left network

In [19]:
comparison_network_incel = edgelist[edgelist['link']=='incel'].iloc[:,:3] #get edgelist only incel
comparison_network_left = edgelist[edgelist['link']=='left'].iloc[:,:3] #get edgelist only left
comparison_network_right = edgelist[edgelist['link']=='right'].iloc[:,:3] #get edgelist only right

G_incel = ig.Graph.TupleList(comparison_network_incel.itertuples(index=False), directed=True, weights=True)
G_left = ig.Graph.TupleList(comparison_network_left.itertuples(index=False), directed=True, weights=True)
G_right = ig.Graph.TupleList(comparison_network_right.itertuples(index=False), directed=True, weights=True)

In [20]:
deg_dict_incel = dict(zip(G_incel.vs['name'], G_incel.degree())) 
deg_dict_left = dict(zip(G_left.vs['name'], G_left.degree()))
deg_dict_right = dict(zip(G_right.vs['name'], G_right.degree())) 

dict_tot = {
    'incel':deg_dict_incel,
    'left':deg_dict_left,
    'right':deg_dict_right
}

In [21]:
def get_weights(x, key):
    """
    x: node id,
    key: 'left', 'right' or 'incel'

    Get the degree of x in the correspondent network
    """

    return dict_tot[key][x]

In [22]:
get_weights = np.vectorize(get_weights) #vectorize function so we can transfom the whole array in a sequence of weights

In [23]:
get_weights(common_nodes, key='incel').sum() / get_weights(nodes_incel, key='incel').sum() * 100
#34% of the nodes degrees in the incel network is composed by nodes that are in all three networks

33.71821484172289

#### incel-extreme right

get how many nodes in the incel network are also in the right network and ARE NOT in the left one
(using nodes degree in incel network as weights)

In [24]:
filter_nodes = (np.isin(nodes_incel, nodes_right) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_right and AREN'T in all three networks

In [25]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#19.3% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme right one


print(f"Right weights : {get_weights(nodes_incel_filtered, 'right').sum() / get_weights(nodes_right, 'right').sum() * 100} %")
#the total weight of the same nodes in the right network is slightly lower, 18.4%

Incel weights : 19.382459782044627 %
Right weights : 18.425324675324674 %


### incel-extreme left

In [30]:
filter_nodes = (np.isin(nodes_incel, nodes_left) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_left and AREN'T in all three networks

In [31]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#11.22% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme left one

print(f"Left weights : {get_weights(nodes_incel_filtered, 'left').sum() / get_weights(nodes_left, 'left').sum() * 100} %")
#the total weight of the same nodes in the left network is lower, 6%

Incel weights : 11.228593668915412 %
Left weights : 5.99011939069576 %


### comparison on outdegree

In [39]:
nodes_incel = edgelist[edgelist['link']=='incel'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_incel = nodes_incel.values.reshape((-1)) #flatten array

nodes_incel = np.unique(nodes_incel) #get unique values

In [40]:
nodes_right = edgelist[edgelist['link']=='right'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_right = nodes_right.values.reshape((-1)) #flatten array

nodes_right = np.unique(nodes_right) #get unique values

In [41]:
nodes_left = edgelist[edgelist['link']=='left'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_left = nodes_left.values.reshape((-1)) #flatten array

nodes_left = np.unique(nodes_left) #get unique values

filter nodes common to all three

In [42]:
nodes_step_1 = nodes_incel[np.isin(nodes_incel, nodes_right)] #get nodes of incel network that are ALSO in right network
common_nodes = nodes_step_1[np.isin(nodes_step_1, nodes_left)] #get nodes of intermediate step (present in both right and incel network) that are ALSO in left network

In [43]:
comparison_network_incel = edgelist[edgelist['link']=='incel'].iloc[:,:3] #get edgelist only incel
comparison_network_left = edgelist[edgelist['link']=='left'].iloc[:,:3] #get edgelist only left
comparison_network_right = edgelist[edgelist['link']=='right'].iloc[:,:3] #get edgelist only right

G_incel = ig.Graph.TupleList(comparison_network_incel.itertuples(index=False), directed=True, weights=True)
G_left = ig.Graph.TupleList(comparison_network_left.itertuples(index=False), directed=True, weights=True)
G_right = ig.Graph.TupleList(comparison_network_right.itertuples(index=False), directed=True, weights=True)

In [44]:
deg_dict_incel = dict(zip(G_incel.vs['name'], G_incel.degree(mode='out'))) 
deg_dict_left = dict(zip(G_left.vs['name'], G_left.degree(mode='out')))
deg_dict_right = dict(zip(G_right.vs['name'], G_right.degree(mode='out'))) 

dict_tot = {
    'incel':deg_dict_incel,
    'left':deg_dict_left,
    'right':deg_dict_right
}

In [45]:
def get_weights(x, key):
    """
    x: node id,
    key: 'left', 'right' or 'incel'

    Get the degree of x in the correspondent network
    """

    return dict_tot[key][x]

In [46]:
get_weights = np.vectorize(get_weights) #vectorize function so we can transfom the whole array in a sequence of weights

In [47]:
get_weights(common_nodes, key='incel').sum() / get_weights(nodes_incel, key='incel').sum() * 100
#34% of the nodes degrees in the incel network is composed by nodes that are in all three networks

30.11157239231967

### incel-extreme right

get how many nodes in the incel network are also in the right network and ARE NOT in the left one
(using nodes degree in incel network as weights)

In [48]:
filter_nodes = (np.isin(nodes_incel, nodes_right) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_right and AREN'T in all three networks

In [49]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#19.6% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme right one


print(f"Right weights : {get_weights(nodes_incel_filtered, 'right').sum() / get_weights(nodes_right, 'right').sum() * 100} %")
#the total weight of the same nodes in the right network is slightly lower, 16.4%

Incel weights : 19.603009859885834 %
Right weights : 16.417748917748916 %


### incel-extreme left

In [50]:
filter_nodes = (np.isin(nodes_incel, nodes_left) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_left and AREN'T in all three networks

In [51]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#11.22% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme left one

print(f"Left weights : {get_weights(nodes_incel_filtered, 'left').sum() / get_weights(nodes_left, 'left').sum() * 100} %")
#the total weight of the same nodes in the left network is lower, 6%

Incel weights : 8.393876491956409 %
Left weights : 6.86496500617538 %


### comparison on indegree

In [52]:
nodes_incel = edgelist[edgelist['link']=='incel'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_incel = nodes_incel.values.reshape((-1)) #flatten array

nodes_incel = np.unique(nodes_incel) #get unique values

In [53]:
nodes_right = edgelist[edgelist['link']=='right'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_right = nodes_right.values.reshape((-1)) #flatten array

nodes_right = np.unique(nodes_right) #get unique values

In [54]:
nodes_left = edgelist[edgelist['link']=='left'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_left = nodes_left.values.reshape((-1)) #flatten array

nodes_left = np.unique(nodes_left) #get unique values

filter nodes common to all three

In [55]:
nodes_step_1 = nodes_incel[np.isin(nodes_incel, nodes_right)] #get nodes of incel network that are ALSO in right network
common_nodes = nodes_step_1[np.isin(nodes_step_1, nodes_left)] #get nodes of intermediate step (present in both right and incel network) that are ALSO in left network

In [56]:
comparison_network_incel = edgelist[edgelist['link']=='incel'].iloc[:,:3] #get edgelist only incel
comparison_network_left = edgelist[edgelist['link']=='left'].iloc[:,:3] #get edgelist only left
comparison_network_right = edgelist[edgelist['link']=='right'].iloc[:,:3] #get edgelist only right

G_incel = ig.Graph.TupleList(comparison_network_incel.itertuples(index=False), directed=True, weights=True)
G_left = ig.Graph.TupleList(comparison_network_left.itertuples(index=False), directed=True, weights=True)
G_right = ig.Graph.TupleList(comparison_network_right.itertuples(index=False), directed=True, weights=True)

In [57]:
deg_dict_incel = dict(zip(G_incel.vs['name'], G_incel.degree(mode='in'))) 
deg_dict_left = dict(zip(G_left.vs['name'], G_left.degree(mode='in')))
deg_dict_right = dict(zip(G_right.vs['name'], G_right.degree(mode='in'))) 

dict_tot = {
    'incel':deg_dict_incel,
    'left':deg_dict_left,
    'right':deg_dict_right
}

In [58]:
def get_weights(x, key):
    """
    x: node id,
    key: 'left', 'right' or 'incel'

    Get the degree of x in the correspondent network
    """

    return dict_tot[key][x]

In [59]:
get_weights = np.vectorize(get_weights) #vectorize function so we can transfom the whole array in a sequence of weights

In [60]:
get_weights(common_nodes, key='incel').sum() / get_weights(nodes_incel, key='incel').sum() * 100
#34% of the nodes degrees in the incel network is composed by nodes that are in all three networks

37.3248572911261

### incel-extreme right

get how many nodes in the incel network are also in the right network and ARE NOT in the left one
(using nodes degree in incel network as weights)

In [61]:
filter_nodes = (np.isin(nodes_incel, nodes_right) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_right and AREN'T in all three networks

In [62]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#19.1% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme right one


print(f"Right weights : {get_weights(nodes_incel_filtered, 'right').sum() / get_weights(nodes_right, 'right').sum() * 100} %")
#the total weight of the same nodes in the right network is slightly higher, 20.4%

Incel weights : 19.161909704203424 %
Right weights : 20.432900432900432 %


### incel-extreme left

In [63]:
filter_nodes = (np.isin(nodes_incel, nodes_left) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_left and AREN'T in all three networks

In [64]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#14.06% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme left one

print(f"Left weights : {get_weights(nodes_incel_filtered, 'left').sum() / get_weights(nodes_left, 'left').sum() * 100} %")
#the total weight of the same nodes in the left network is lower, 5.11%

Incel weights : 14.063310845874415 %
Left weights : 5.115273775216139 %


---------

# Comparison on two iterations

In [86]:
edgelist_incel = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./incel*/*_edgelist/[f,s]*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_incel = pd.concat([edgelist_incel,df])

In [87]:
edgelist_incel['link'] = ['incel']*len(edgelist_incel)

In [88]:
edgelist_right = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./*right*/*_edgelist/[f,s]*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_right = pd.concat([edgelist_right,df])

In [89]:
edgelist_right['link'] = ['right']*len(edgelist_right)

In [90]:
edgelist_left = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('./*left*/*_edgelist/[f,s]*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist_left = pd.concat([edgelist_left,df])

In [91]:
edgelist_left['link'] = ['left']*len(edgelist_left)

In [92]:
edgelist = pd.concat([edgelist_incel, edgelist_left, edgelist_right])

## Visualization

### Incel and extreme right

In [94]:
edgelist_ir = edgelist[(edgelist['link']=='incel') | (edgelist['link']=='right')]

In [95]:
G = ig.Graph.TupleList(edgelist_ir.iloc[:,:3].itertuples(index=False), directed=True, weights=True)

In [96]:
## community detection
communities = G.as_undirected().community_leiden(objective_function="modularity")

# add community as node label
G.vs['community'] = communities.membership

In [97]:
G.vs['desc'] = list(map(lambda x: id_to_desc.get(int(x),x), G.vs['name']))
G.vs['label'] = list(map(lambda x: id_to_names.get(int(x),x), G.vs['name']))

In [98]:
Sigma(G, node_size=G.degree(mode='out'),
      node_color=G.vs['community'], 
      hide_edges_on_move=True,
      edge_color=edgelist_ir['link'].tolist(),
      node_color_gradient='Viridis', node_label=G.vs['label'],
     layout_settings = {"barnesHutOptimize":False,
                       'gravity': 0.1})

Sigma(ig.Graph with 619 nodes and 1,080 edges)

### incel and extreme left

In [99]:
edgelist_il = edgelist[(edgelist['link']=='incel') | (edgelist['link']=='left')]

In [100]:
G = ig.Graph.TupleList(edgelist_il.iloc[:,:3].itertuples(index=False), directed=True, weights=True)

In [101]:
## community detection
communities = G.as_undirected().community_leiden(objective_function="modularity")

# add community as node label
G.vs['community'] = communities.membership

In [102]:
G.vs['desc'] = list(map(lambda x: id_to_desc.get(int(x),x), G.vs['name']))
G.vs['label'] = list(map(lambda x: id_to_names.get(int(x),x), G.vs['name']))

In [103]:
Sigma(G, node_size=G.degree(mode='out'),
      node_color=G.vs['community'], 
      hide_edges_on_move=True,
      edge_color=edgelist['link'].tolist(),
      node_color_gradient='Viridis', node_label=G.vs['label'],
     layout_settings = {"barnesHutOptimize":False,
                       'gravity': 0.1})

Sigma(ig.Graph with 909 nodes and 1,729 edges)

## comparison on total degree

In [104]:
nodes_incel = edgelist[edgelist['link']=='incel'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_incel = nodes_incel.values.reshape((-1)) #flatten array

nodes_incel = np.unique(nodes_incel) #get unique values

In [105]:
nodes_right = edgelist[edgelist['link']=='right'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_right = nodes_right.values.reshape((-1)) #flatten array

nodes_right = np.unique(nodes_right) #get unique values

In [106]:
nodes_left = edgelist[edgelist['link']=='left'].iloc[:,:2] #gets only forward_from and forward_to columns

nodes_left = nodes_left.values.reshape((-1)) #flatten array

nodes_left = np.unique(nodes_left) #get unique values

filter nodes common to all three

In [107]:
nodes_step_1 = nodes_incel[np.isin(nodes_incel, nodes_right)] #get nodes of incel network that are ALSO in right network
common_nodes = nodes_step_1[np.isin(nodes_step_1, nodes_left)] #get nodes of intermediate step (present in both right and incel network) that are ALSO in left network

In [108]:
comparison_network_incel = edgelist[edgelist['link']=='incel'].iloc[:,:3] #get edgelist only incel
comparison_network_left = edgelist[edgelist['link']=='left'].iloc[:,:3] #get edgelist only left
comparison_network_right = edgelist[edgelist['link']=='right'].iloc[:,:3] #get edgelist only right

G_incel = ig.Graph.TupleList(comparison_network_incel.itertuples(index=False), directed=True, weights=True)
G_left = ig.Graph.TupleList(comparison_network_left.itertuples(index=False), directed=True, weights=True)
G_right = ig.Graph.TupleList(comparison_network_right.itertuples(index=False), directed=True, weights=True)

In [109]:
deg_dict_incel = dict(zip(G_incel.vs['name'], G_incel.degree())) 
deg_dict_left = dict(zip(G_left.vs['name'], G_left.degree()))
deg_dict_right = dict(zip(G_right.vs['name'], G_right.degree())) 

dict_tot = {
    'incel':deg_dict_incel,
    'left':deg_dict_left,
    'right':deg_dict_right
}

In [110]:
def get_weights(x, key):
    """
    x: node id,
    key: 'left', 'right' or 'incel'

    Get the degree of x in the correspondent network
    """

    return dict_tot[key][x]

In [111]:
get_weights = np.vectorize(get_weights) #vectorize function so we can transfom the whole array in a sequence of weights

In [112]:
get_weights(common_nodes, key='incel').sum() / get_weights(nodes_incel, key='incel').sum() * 100
#1.5% of the nodes degrees in the incel network is composed by nodes that are in all three networks

1.5909090909090908

### incel-extreme right

get how many nodes in the incel network are also in the right network and ARE NOT in the left one
(using nodes degree in incel network as weights)

In [113]:
filter_nodes = (np.isin(nodes_incel, nodes_right) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_right and AREN'T in all three networks

In [114]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#19.3% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme right one


print(f"Right weights : {get_weights(nodes_incel_filtered, 'right').sum() / get_weights(nodes_right, 'right').sum() * 100} %")
#the total weight of the same nodes in the right network is slightly lower, 18.4%

Incel weights : 3.295454545454545 %
Right weights : 2.734375 %


### incel-extreme left

In [115]:
filter_nodes = (np.isin(nodes_incel, nodes_left) & ~np.isin(nodes_incel, common_nodes)) 
nodes_incel_filtered = nodes_incel[filter_nodes] #get all nodes in nodes_incel that ARE in nodes_left and AREN'T in all three networks

In [116]:
print(f"Incel weights : {get_weights(nodes_incel_filtered, 'incel').sum() / get_weights(nodes_incel, 'incel').sum() * 100} %")
#11.22% of the total of nodes degrees in the incel network is composed by nodes that are in the extreme left one

print(f"Left weights : {get_weights(nodes_incel_filtered, 'left').sum() / get_weights(nodes_left, 'left').sum() * 100} %")
#the total weight of the same nodes in the left network is lower, 6%

Incel weights : 4.090909090909091 %
Left weights : 0.3878975950349108 %
